In [1]:
import yfinance as yf
import datetime
import time

aapl = yf.Ticker('AAPL')

In [16]:
aapl.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 147000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [20]:
aapl.info['sector']

'Technology'

In [9]:
# get current date and date 60 days ago
current_date = datetime.datetime.now()
current_date = current_date.strftime("%Y-%m-%d")
sixty_days_ago = (datetime.datetime.now() - datetime.timedelta(days=59))
sixty_days_ago = sixty_days_ago.strftime('%Y-%m-%d')

aapl.history(interval='15m', auto_adjust=True, start=sixty_days_ago, end=current_date)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-02-08 09:30:00-05:00,136.029999,136.250000,134.940094,135.736206,10408110,0,0
2021-02-08 09:45:00-05:00,135.625000,135.800003,135.360001,135.690002,4316546,0,0
2021-02-08 10:00:00-05:00,135.679993,136.309998,135.600006,135.958405,3860834,0,0
2021-02-08 10:15:00-05:00,135.960007,136.220001,135.720993,135.910004,2529242,0,0
2021-02-08 10:30:00-05:00,135.909393,136.630005,135.880005,136.590805,2967968,0,0
...,...,...,...,...,...,...,...
2021-04-05 14:45:00-04:00,125.848503,125.910004,125.599998,125.629997,1875123,0,0
2021-04-05 15:00:00-04:00,125.635002,125.720001,125.464996,125.628403,2353948,0,0
2021-04-05 15:15:00-04:00,125.625000,125.820000,125.480003,125.549004,2312180,0,0


In [2]:
# import SQLite

import sqlite3
from sqlite3 import Error

conn = sqlite3.connect('stockPrediction.db')

In [13]:
# build SQLite tables

# build table if not exists

# table for sectors, industries
conn.execute('''CREATE TABLE IF NOT EXISTS sector (
    sector_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    sector TEXT NOT NULL
);
''')

conn.execute('''CREATE TABLE IF NOT EXISTS industry (
    industry_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    industry TEXT NOT NULL
);
''')

# stock table
conn.execute('''CREATE TABLE IF NOT EXISTS stock (
    stock_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    stock_symbol TEXT NOT NULL,
    short_name TEXT,
    full_name TEXT,
    sector_id TEXT,
    industry_id TEXT,
    datetime_added TEXT NOT NULL,
    FOREIGN KEY (sector_id)
        REFERENCES sector (sector_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (industry_ID)
        REFERENCES industry (industry_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE
);''')

conn.execute('''CREATE TABLE IF NOT EXISTS price_history (
    price_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    stock_id INTEGER NOT NULL,
    price_datetime TEXT NOT NULL,
    open_price REAL NOT NULL,
    high_price REAL NOT NULL,
    low_price REAL NOT NULL,
    close_price REAL NOT NULL,
    volume INTEGER NOT NULL,
    dividends REAL,
    stock_splits TEXT,
    datetime_added TEXT NOT NULL,
    FOREIGN KEY (stock_id)
        REFERENCES stock (stock_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE
);
''')



## Add the stocks from a text file to the database, and get info regarding their sectors, industries, etc.

In [27]:
from stock import Stock
from stock import Price
from stock import get_current_datetime
import sqlite3
from sqlite3 import Error
import yfinance as yf
import datetime
import time

conn = sqlite3.connect('stockPrediction.db')

# open a list of symbols, each symbol as a single line
symbols = open('allUnderFiveDollarsAndPrevOwned - Copy.txt')

dbq = conn.execute('SELECT stock_symbol FROM stock')
dbq = dbq.fetchall()

db_symbols = [x[0] for x in dbq]

# loop thru the symbols
for symbol in symbols:

    symbol = symbol.strip('\n')

    if symbol in db_symbols:
        continue

    # call to yf to get the data
    data = yf.Ticker(symbol)

    # take the data and create a new Stock object w/ it, just to keep organized and help myself w/ OOP
    # try/except block for symbols that have since been delisted
    try:
        symbol = Stock(symbol, short_name=data.info['shortName'], full_name=data.info['longName'], sector=data.info['sector'], industry=data.info['industry'])
    except KeyError:
        print(f'{symbol} no longer a listed security')
        continue

    # trying this to see if I can correct my db locked error:
    conn.commit()

    # check to see if this security's industry/sector are already in the DB, if not, add them
    # must have a DB connection open prior to running loop
    symbol.check_industry(symbol.industry, conn)
    symbol.check_sector(symbol.sector, conn)

    # remove rogue single quotes from the short/full names
    symbol.short_name = symbol.short_name.replace("'", "")
    symbol.full_name = symbol.full_name.replace("'", "")

    # add the symbol to the db
    conn.execute(f"INSERT INTO stock (stock_symbol, short_name, full_name, sector_id, industry_id, datetime_added) VALUES ('{symbol.symbol}', '{symbol.short_name}', '{symbol.full_name}', {symbol.sector_id}, {symbol.industry_id}, '{get_current_datetime()}')")
    conn.commit()
    print(f'Wrote {symbol.symbol} to the DB')

    # pause to not get us blocked by yf
    time.sleep(3)
print('Done.')

the DB
Wrote MMJFF to the DB
Wrote MMLP to the DB
Wrote MMMB to the DB
Wrote MMND to the DB
Wrote MMNFF to the DB
Wrote MMRGF to the DB
Wrote MMSDF to the DB
Wrote MMTIF to the DB
Wrote MMTV to the DB
Wrote MNDJF to the DB
Wrote MNDO to the DB
Wrote MNKD to the DB
Wrote MNRLF to the DB
Wrote MNTR to the DB
Wrote MNZO to the DB
Wrote MODC to the DB
Wrote MOGO to the DB
Wrote MOGU to the DB
Wrote MOHO to the DB
Wrote MOST to the DB
Wrote MOSY to the DB
Wrote MOTS to the DB
Wrote MOXC to the DB
Wrote MPIR to the DB
Wrote MPXOF to the DB
Wrote MRAM to the DB
Wrote MREO to the DB
Wrote MRIN to the DB
Wrote MRKR to the DB
Wrote MRMD to the DB
MRQRF no longer a listed security
Wrote MRZM to the DB
Wrote MSLM to the DB
Wrote MSN to the DB
Wrote MSTF to the DB
Wrote MTC to the DB
Wrote MTENY to the DB
Wrote MTL to the DB
Wrote MTL/P to the DB
Wrote MTLFF to the DB
Wrote MTNB to the DB
Wrote MTP to the DB
Wrote MTR to the DB
Wrote MTRO to the DB
Wrote MTSL to the DB
Wrote MUFG to the DB
Wrote MU

In [24]:
symbol.short_name

SyntaxError: invalid syntax (<ipython-input-24-97b9a5578b4e>, line 1)

In [12]:
# this was to test to see if i wrote my list comprehension correctly

from stock import Stock
from stock import Price
from stock import get_current_datetime
import sqlite3
from sqlite3 import Error
import yfinance as yf
import datetime
import time

conn = sqlite3.connect('stockPrediction.db')

# open a list of symbols, each symbol as a single line
symbols = open('allUnderFiveDollarsAndPrevOwned - Copy.txt')

dbq = conn.execute('SELECT stock_symbol FROM stock')
dbq = dbq.fetchall()

db_symbols = [x[0] for x in dbq]

In [8]:
get_current_datetime()

'2021-04-10 13:24:56"'

In [2]:
# this is me testing my stock class
from stock import Stock
from stock import Price

aapl = Stock('AAPL')


In [3]:
print(aapl.symbol)

AAPL


In [4]:
aapl.short_name = 'Apple Inc.'

In [5]:
print(aapl.short_name)

Apple Inc.


In [10]:
symbol.symbol

'BLPH'

In [18]:
symbol.sector_id

(1,)

In [1]:
# testing class methods for stock
from stock import Stock
import yfinance as yf
import sqlite3

conn = sqlite3.connect('stockPrediction.db')

data = yf.Ticker('AAPL')

aapl = Stock(data.info['symbol'])
aapl.industry = data.info['industry']
aapl.sector = data.info['sector']

In [8]:
aapl.industry

'Consumer Electronics'

In [9]:
aapl.sector

'Technology'

In [2]:
aapl.check_industry(aapl.industry, conn)

In [3]:
print(aapl.industry_id)

(10,)


In [1]:
import yfinance as yf
import datetime

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
sixty_days_ago = (datetime.datetime.now() - datetime.timedelta(days=59)).strftime("%Y-%m-%d")

data = yf.Ticker("BEBE")

data.history(
    start = sixty_days_ago,
    end = current_date,
    interval = '15m',
    auto_adjust = True
)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-02-12 09:30:00-05:00,4.55,4.55,4.55,4.55,24.0,0.0,0
2021-02-16 11:00:00-05:00,5.39,5.39,5.39,5.39,48.0,0.0,0
2021-02-16 13:45:00-05:00,4.85,4.85,4.55,4.55,2914.0,0.0,0
2021-02-17 09:30:00-05:00,4.55,4.55,4.55,4.55,0.0,0.0,0
2021-02-17 11:00:00-05:00,5.35,5.35,5.35,5.35,258.0,0.0,0
...,...,...,...,...,...,...,...
2021-04-06 15:45:00-04:00,5.20,5.20,5.20,5.20,185.0,0.0,0
2021-04-07 11:00:00-04:00,5.10,5.10,5.10,5.10,112.0,0.0,0
2021-04-07 14:15:00-04:00,5.10,5.10,5.10,5.10,2226.0,0.0,0


In [15]:
import pandas as pd

data = "INSERT INTO price_history (stock_id, price_datetime, open_price, high_price, low_price, close_price, volume, dividends, stock_splits, datetime_added) VALUES \n(229, '2021-02-12 09:30:00', 4.550000190734863, 4.550000190734863, 4.550000190734863, 4.550000190734863, 24.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-16 11:00:00', 5.389999866485596, 5.389999866485596, 5.389999866485596, 5.389999866485596, 48.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-16 13:45:00', 4.849999904632568, 4.849999904632568, 4.550000190734863, 4.550000190734863, 2914.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-17 09:30:00', 4.550000190734863, 4.550000190734863, 4.550000190734863, 4.550000190734863, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-17 11:00:00', 5.349999904632568, 5.349999904632568, 5.349999904632568, 5.349999904632568, 258.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-17 12:30:00', 5.25, 5.349999904632568, 5.25, 5.349999904632568, 3480.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-17 12:45:00', 4.550000190734863, 4.550000190734863, 4.550000190734863, 4.550000190734863, 203.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-18 09:30:00', 4.550000190734863, 4.550000190734863, 4.550000190734863, 4.550000190734863, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-18 11:15:00', 5.349999904632568, 5.989999771118164, 5.349999904632568, 5.989999771118164, 2110.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-18 11:30:00', 5.369999885559082, 5.369999885559082, 4.599999904632568, 4.599999904632568, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-19 09:30:00', 4.550000190734863, 4.550000190734863, 4.550000190734863, 4.550000190734863, 107.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-19 10:15:00', 5.980000019073486, 5.980000019073486, 5.980000019073486, 5.980000019073486, 402.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-19 12:45:00', 5.5, 5.5, 5.5, 5.5, 173.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-19 15:45:00', 5.5, 5.5, 5.5, 5.5, 227.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-22 09:30:00', 5.5, 5.5, 5.5, 5.5, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-22 13:00:00', 5.449999809265137, 5.449999809265137, 5.449999809265137, 5.449999809265137, 200.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-23 09:30:00', 5.5, 5.5, 5.5, 5.5, 175.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-23 09:45:00', 5.640100002288818, 5.640100002288818, 5.640100002288818, 5.640100002288818, 100.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-23 10:15:00', 5.679999828338623, 5.679999828338623, 5.679999828338623, 5.679999828338623, 200.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-23 10:45:00', 5.360000133514404, 5.360000133514404, 5.360000133514404, 5.360000133514404, 926.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-23 15:45:00', 5.300000190734863, 5.300000190734863, 5.289999961853027, 5.289999961853027, 2011.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-24 09:30:00', nan, nan, nan, nan, nan, 0.06, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-24 14:45:00', 5.75, 5.75, 5.75, 5.75, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-24 15:45:00', 4.699999809265137, 5.289999961853027, 4.699999809265137, 5.289999961853027, 256.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-25 09:30:00', 4.699999809265137, 5.75, 4.699999809265137, 5.75, 330.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 09:30:00', 5.809999942779541, 5.809999942779541, 5.809999942779541, 5.809999942779541, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 11:00:00', 4.71999979019165, 4.71999979019165, 4.5, 4.5, 500.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 11:15:00', 4.820000171661377, 4.820000171661377, 4.820000171661377, 4.820000171661377, 110.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 12:30:00', 4.849999904632568, 4.849999904632568, 4.849999904632568, 4.849999904632568, 100.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 12:45:00', 5.639999866485596, 5.639999866485596, 5.639999866485596, 5.639999866485596, 101.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 14:30:00', 4.849999904632568, 4.849999904632568, 4.849999904632568, 4.849999904632568, 150.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-02-26 15:45:00', 5.639999866485596, 5.639999866485596, 5.639999866485596, 5.639999866485596, 157.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-01 09:30:00', 5.639999866485596, 5.639999866485596, 5.639999866485596, 5.639999866485596, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-01 13:00:00', 5.570000171661377, 5.650000095367432, 5.570000171661377, 5.650000095367432, 2110.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-01 14:00:00', 5.010000228881836, 5.010000228881836, 5.010000228881836, 5.010000228881836, 115.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-01 15:45:00', 4.760000228881836, 4.760000228881836, 4.760000228881836, 4.760000228881836, 100.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-02 09:30:00', 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-02 10:00:00', 5.0, 5.0, 5.0, 5.0, 141.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-02 10:45:00', 5.0, 5.0, 5.0, 5.0, 150.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-02 13:45:00', 5.150000095367432, 5.150000095367432, 4.550000190734863, 4.550000190734863, 6372.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-02 15:45:00', 5.440000057220459, 5.440000057220459, 5.440000057220459, 5.440000057220459, 113.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-03 14:30:00', 5.739999771118164, 5.75, 5.739999771118164, 5.75, 216.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-03 14:45:00', 5.75, 5.75, 5.75, 5.75, 126.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-03 15:45:00', 5.150000095367432, 5.150000095367432, 5.150000095367432, 5.150000095367432, 428.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 13:15:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 134.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 14:15:00', 5.599999904632568, 5.599999904632568, 5.599999904632568, 5.599999904632568, 4906.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 14:30:00', 5.599999904632568, 5.599999904632568, 5.599999904632568, 5.599999904632568, 1200.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 14:45:00', 5.599999904632568, 5.599999904632568, 5.599999904632568, 5.599999904632568, 3906.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 15:00:00', 5.559999942779541, 5.559999942779541, 5.559999942779541, 5.559999942779541, 110.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 15:30:00', 5.699999809265137, 5.699999809265137, 5.699999809265137, 5.699999809265137, 505.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-04 15:45:00', 5.699999809265137, 5.699999809265137, 5.699999809265137, 5.699999809265137, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-05 09:30:00', 5.690000057220459, 5.690000057220459, 5.690000057220459, 5.690000057220459, 683.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-05 14:00:00', 5.599999904632568, 5.599999904632568, 5.599999904632568, 5.599999904632568, 335.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-05 14:30:00', 5.739999771118164, 5.739999771118164, 5.739999771118164, 5.739999771118164, 181.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-08 09:30:00', 5.599999904632568, 5.599999904632568, 5.599999904632568, 5.599999904632568, 3808.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-08 11:45:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 210.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-09 09:30:00', 5.789999961853027, 5.789999961853027, 5.789999961853027, 5.789999961853027, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-09 11:45:00', 5.75, 5.75, 5.75, 5.75, 210.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-09 14:45:00', 5.650000095367432, 5.650000095367432, 5.650000095367432, 5.650000095367432, 420.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-09 15:15:00', 5.699999809265137, 5.739999771118164, 5.699999809265137, 5.739999771118164, 494.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-10 10:30:00', 5.769999980926514, 5.769999980926514, 5.769999980926514, 5.769999980926514, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-10 10:45:00', 5.769999980926514, 5.769999980926514, 5.769999980926514, 5.769999980926514, 820.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-10 11:00:00', 5.78000020980835, 5.78000020980835, 5.78000020980835, 5.78000020980835, 751.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-10 11:15:00', 5.78000020980835, 5.789999961853027, 5.769999980926514, 5.789999961853027, 4285.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-10 11:30:00', 5.800000190734863, 5.800000190734863, 5.800000190734863, 5.800000190734863, 311.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 09:30:00', 5.900000095367432, 5.900000095367432, 5.900000095367432, 5.900000095367432, 199.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 09:45:00', 5.855000019073486, 5.855000019073486, 5.730000019073486, 5.855000019073486, 214.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 11:00:00', 5.699999809265137, 5.800000190734863, 5.699999809265137, 5.800000190734863, 833.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 11:15:00', 5.980000019073486, 5.980000019073486, 5.980000019073486, 5.980000019073486, 103.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 12:30:00', 5.960000038146973, 5.960000038146973, 5.960000038146973, 5.960000038146973, 102.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 13:15:00', 5.728000164031982, 5.728000164031982, 5.728000164031982, 5.728000164031982, 109.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-11 15:15:00', 5.699999809265137, 5.840000152587891, 5.699999809265137, 5.75, 9467.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-12 10:15:00', 5.630000114440918, 5.630000114440918, 5.630000114440918, 5.630000114440918, 11.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-12 11:30:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 3010.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-15 09:30:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 2.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-15 14:45:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 100.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-16 10:45:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 795.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-16 13:00:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 750.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-16 14:15:00', 5.550000190734863, 5.550000190734863, 5.550000190734863, 5.550000190734863, 5089.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-17 09:30:00', 5.550000190734863, 5.900000095367432, 5.550000190734863, 5.900000095367432, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-18 09:45:00', 5.889999866485596, 5.889999866485596, 5.889999866485596, 5.889999866485596, 161.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-22 09:30:00', 5.880000114440918, 5.880000114440918, 5.880000114440918, 5.880000114440918, 90.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-23 09:30:00', 5.869999885559082, 5.869999885559082, 5.869999885559082, 5.869999885559082, 230.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-25 09:30:00', 5.800000190734863, 5.800000190734863, 5.800000190734863, 5.800000190734863, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-25 11:15:00', 5.409999847412109, 5.409999847412109, 4.760000228881836, 4.760000228881836, 500.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-25 13:15:00', 5.739999771118164, 5.739999771118164, 5.739999771118164, 5.739999771118164, 110.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-26 09:30:00', 5.800000190734863, 5.800000190734863, 5.800000190734863, 5.800000190734863, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-26 11:15:00', 5.739999771118164, 5.739999771118164, 5.739999771118164, 5.739999771118164, 121.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-30 09:30:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-30 14:15:00', 5.489999771118164, 5.489999771118164, 5.489999771118164, 5.489999771118164, 271.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-30 15:30:00', 5.489999771118164, 5.489999771118164, 5.489999771118164, 5.489999771118164, 813.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-31 09:30:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 25.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-31 12:00:00', 5.800000190734863, 5.800000190734863, 5.800000190734863, 5.800000190734863, 192.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-31 12:45:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 140.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-03-31 14:30:00', 5.300000190734863, 5.300000190734863, 5.300000190734863, 5.300000190734863, 135.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-01 09:30:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-01 10:45:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 101.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-01 15:30:00', 5.199999809265137, 5.199999809265137, 5.150000095367432, 5.150000095367432, 306.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-05 09:30:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-05 13:15:00', 5.199999809265137, 5.199999809265137, 5.199999809265137, 5.199999809265137, 374.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-06 13:00:00', 5.25, 5.25, 5.25, 5.25, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-06 15:30:00', 5.25, 5.25, 5.25, 5.25, 300.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-06 15:45:00', 5.199999809265137, 5.199999809265137, 5.199999809265137, 5.199999809265137, 185.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-07 11:00:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 112.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-07 14:15:00', 5.099999904632568, 5.099999904632568, 5.099999904632568, 5.099999904632568, 2226.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-08 12:15:00', 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, '0', '2021-04-12 19:56:50' ),\n(229, '2021-04-09 15:30:00', 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, '0', '2021-04-12 19:56:50' );"

vals = data.splitlines()
vals = [i.split(",") for i in vals]
vals

#df = pd.DataFrame(vals, columns=['stock_id', 'price_datetime', 'open', 'high', 'low', 'close', 'volume', 'div.', 'stock split', 'datetime recorded'])
#df

863',
  ' 0.0',
  ' 0.0',
  " '0'",
  " '2021-04-12 19:56:50' )",
  ''],
 ['(229',
  " '2021-02-17 11:00:00'",
  ' 5.349999904632568',
  ' 5.349999904632568',
  ' 5.349999904632568',
  ' 5.349999904632568',
  ' 258.0',
  ' 0.0',
  " '0'",
  " '2021-04-12 19:56:50' )",
  ''],
 ['(229',
  " '2021-02-17 12:30:00'",
  ' 5.25',
  ' 5.349999904632568',
  ' 5.25',
  ' 5.349999904632568',
  ' 3480.0',
  ' 0.0',
  " '0'",
  " '2021-04-12 19:56:50' )",
  ''],
 ['(229',
  " '2021-02-17 12:45:00'",
  ' 4.550000190734863',
  ' 4.550000190734863',
  ' 4.550000190734863',
  ' 4.550000190734863',
  ' 203.0',
  ' 0.0',
  " '0'",
  " '2021-04-12 19:56:50' )",
  ''],
 ['(229',
  " '2021-02-18 09:30:00'",
  ' 4.550000190734863',
  ' 4.550000190734863',
  ' 4.550000190734863',
  ' 4.550000190734863',
  ' 0.0',
  ' 0.0',
  " '0'",
  " '2021-04-12 19:56:50' )",
  ''],
 ['(229',
  " '2021-02-18 11:15:00'",
  ' 5.349999904632568',
  ' 5.989999771118164',
  ' 5.349999904632568',
  ' 5.989999771118164',
  ' 2110